In [1]:
from census import Census
from us import states
import pandas as pd
import numpy as np
import pylab as pl
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
c = Census("4c26aa6ebbaef54a55d3903212eabbb506ade381")

In [3]:
buckets = [0, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 60000, 75000, 
           100000, 125000, 150000, 200000,1000000]

In [4]:
buckets_2 = [0, 10000, 15000, 25000, 35000, 50000, 65000, 75000, 1000000]

In [5]:
var=['B19013_001E','B19013_001M']
for i in range(2,10):
    var.append('B19001_00'+str(i)+'E')
    var.append('B19001_00'+str(i)+'M')
for i in range(10,18):
    var.append('B19001_0'+str(i)+'E')
    var.append('B19001_0'+str(i)+'M')
for i in range(3,10):
    var.append('B07010_00'+str(i)+'E')
    var.append('B07010_00'+str(i)+'M')
for i in range(10,12):
    var.append('B07010_0'+str(i)+'E')
    var.append('B07010_0'+str(i)+'M')

#### Choose the state and insert it as part as the api json

In [6]:
state = '36'

In [7]:
sql_query='state:{} county:*'.format(state)

In [8]:
dic = c.acs5.get(var, geo = {'for': 'tract:*',
                             'in': sql_query})

In [9]:
var.append('county')
var.append('state')
var.append('tract')

In [10]:
df = pd.DataFrame.from_dict(dic)

In [11]:
df = df.reset_index()

In [12]:
df = df.drop(columns = 'index')

#### Exporting the raw file

In [13]:
df.to_csv('variables_' + state + '.csv', index = None)

#### Calculating the thresholds for each of the counties

In [14]:
county = df.county.unique()
counties = {}
counties = counties.fromkeys(county)
for i in county:
    counties[i] = np.median(df[df.county == i].B19013_001E)
df_threshold = pd.DataFrame.from_dict(counties, orient='index')
df_threshold = df_threshold.rename(columns={0:'median'})

In [15]:
#df_typo['vvli_threshold']=0.3*df_typo['median']
df_threshold['vli_threshold'] = 0.5*df_threshold['median']
df_threshold['li_threshold'] = 0.8*df_threshold['median']
df_threshold['mi_threshold'] = 1.2*df_threshold['median']
df_threshold['mhi_threshold'] = 1.5*df_threshold['median']
df_threshold['hi_threshold'] = 2*df_threshold['median']
df_threshold['vhi_threshold'] = 1000000

#### Creating the different typologies based on each county's median income

In [16]:
values=[]
for i in county:
    with open('variables_'+state+'.csv', 'r') as fi:
        reader = csv.DictReader(fi)
        def creator(reader, c):
            bt = filter(lambda x: x['county'] == c, reader)
            return (list(bt))
        value = creator(reader, i)
    values.append(value)

#### Creating the migration dataframe

In [17]:
table_2 = np.zeros((len(county),len(buckets_2)-1))
for j in range(0, len(county)):
    col = 1
    for i in range(0, len(buckets_2)-1):
        if buckets_2[i] > df_threshold.iloc[j, col]:
            table_2[j, i] = 0
            col = col+1
        elif buckets_2[i+1] < df_threshold.iloc[j, col]:
                table_2[j, i] = True
        else:
                table_2[j, i] = (df_threshold.iloc[j, col] - buckets_2[i]) / (buckets_2[i+1] - buckets_2[i])
                col = col+1

In [18]:
census_tract = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets_2)-1):
            moved = float(values[j][i][var[36+col*2]])
            if float(values[j][i][var[34]])!=0:
                per = moved/float(values[j][i][var[34]])
            else:
                per = 0
            if table_2[j, col] == 1:
                summ = summ + per
            elif table_2[j, col] != 0:
                summ = summ + per*table_2[j, col]
                typo.append(summ)
                summ = per*(1-table_2[j, col])
            elif ((table_2[j, col] == 0)&(float(values[j][i][var[34]])!=0)):
                typo.append(summ)
                col = col+1
                summ = per * table_2[j, col] + float(values[j][i][var[36+(col-1)*2]])/float(values[j][i][var[34]]) 
        typo.append(summ)
        census_tract.append(typo)

#### Calculating the margin of error for the migration typologies

In [19]:
mig_moe = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets_2)-1):
            dividend_1 = float(values[j][i][var[37+col*2]])
            divisor_1 = float(values[j][i][var[36+col*2]])
            dividend = float(values[j][i][var[37+(col+1)*2]])
            divisor = float(values[j][i][var[36+(col+1)*2]])
            dividend_2 = float(values[j][i][var[35]])
            divisor_2 = float(values[j][i][var[34]])
            if ((divisor != 0) & (divisor_1 != 0) & (divisor_2 != 0)):
                error = (dividend_1/divisor_1 + dividend_2/divisor_2)*(divisor_1/divisor_2)
                error_2 = (dividend/divisor + dividend_2/divisor_2)*(divisor/divisor_2)
            else:
                error = 0
            if table_2[j, col] == 1:
                summ = summ + error
            elif table_2[j, col] == 0:
                typo.append(summ)
                col = col+1
                summ = summ + error_2*table_2[j, col] + error*table_2[j, col-1]
            else :
                summ = summ + error*table_2[j, col]
                typo.append(summ)
                summ = (1-table_2[j, col])*error
        typo.append(summ)
        mig_moe.append(typo)

In [20]:
mig_moe_df = pd.DataFrame(mig_moe)

In [21]:
mig_moe_df = pd.DataFrame(np.nan_to_num(mig_moe_df))

In [22]:
df_mig = pd.DataFrame(census_tract)

In [23]:
df_mig = pd.DataFrame(np.nan_to_num(df_mig))

In [24]:
df_mig = df_mig.rename(columns={0:'vli_mig_10', 1:'li_mig_10', 2:'mi_mig_10', 3:'mhi_mig_10', 4:'hi_mig_10', 
                                5:'vhi_mig_10'})
mig_moe_df = mig_moe_df.rename(columns={0:'vli_mig_10_moe', 1:'li_mig_10_moe', 2:'mi_mig_10_moe', 3:'mhi_mig_10_moe',
                                        4:'hi_mig_10_moe',5:'vhi_mig_10_moe'})

In [25]:
df_final = pd.concat([df_mig, mig_moe_df, pd.DataFrame(df['state']), pd.DataFrame(df['county']), 
                      pd.DataFrame(df['tract'])], axis=1)

In [26]:
df_final['per_all_li_mig_10'] = df_final['vli_mig_10'] + df_final['li_mig_10']

#### Droping null values

In [27]:
prev_len = len(df_final)

In [28]:
df_final = df_final.dropna(axis = 0, how = 'any')

In [29]:
final_len = len(df_final)

In [30]:
print('The percentage of information lost is: {}'.format((prev_len-final_len)*100/prev_len))

The percentage of information lost is: 0.0


#### Exporting the final file

In [31]:
df_final['GEOid2'] = df_final['state'] + df_final['county'] + df_final['tract']

In [32]:
df_final.to_csv(state + '_2010_variables.csv', index = None)